In [ ]:
#TESTING WEB SCRAPING

#For single recipe, taking nutrient info

In [37]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [162]:
url = 'https://www.allrecipes.com/recipe/18057/sweet-potato-casserole-ii/'
url1 = url + 'fullrecipenutrition'
r = requests.get(url1)
#print('Response body: {0}'.format(r.text))

In [81]:
page_body = r.text
soup = BeautifulSoup(page_body, 'html.parser')

In [82]:
#this is the div containing all the nutrition information
nutrition_info = soup.find_all('div', class_='nutrition-row')

In [163]:
#nutrition_info

In [164]:
nutrient_list = []
recipe = soup.find('h2').text
for n in nutrition_info:
    name = n.find(class_ = 'nutrient-name').text
    amount = n.find(class_ = 'nutrient-value').text
    name = name[:name.index(':')]
    nutrient_list.append({'nutrient': name, 'amount': amount, 'recipe': recipe, 'URL': url})
nutrients = pd.DataFrame(nutrient_list)

nutrients.head()

,URL,amount,nutrient,recipe
0,https://www.allrecipes.com/recipe/18057/sweet-...,15.3g,Total Fat,Sweet Potato Casserole II
1,https://www.allrecipes.com/recipe/18057/sweet-...,7.0g,Saturated Fat,Sweet Potato Casserole II
2,https://www.allrecipes.com/recipe/18057/sweet-...,49mg,Cholesterol,Sweet Potato Casserole II
3,https://www.allrecipes.com/recipe/18057/sweet-...,103mg,Sodium,Sweet Potato Casserole II
4,https://www.allrecipes.com/recipe/18057/sweet-...,337mg,Potassium,Sweet Potato Casserole II


In [ ]:
# & ingredients

In [157]:
s = requests.get(url)

In [158]:
page_body2 = s.text
soup2 = BeautifulSoup(page_body2, 'html.parser')

In [165]:
ingredient_info = soup2.find_all(class_='recipe-container-outer')
#ingredient_info

In [161]:
ingredient_list = []
for i in ingredient_info:
    ingredients = i.find_all('span', {'itemprop':'recipeIngredient'}, class_ = 'recipe-ingred_txt added')
    for x in ingredients:
        ingredient_list.append({'ingredient': x.text, 'URL':url})

ingredients = pd.DataFrame(ingredient_list)

ingredients.head()

,URL,ingredient
0,https://www.allrecipes.com/recipe/18057/sweet-...,4 1/2 cups cooked and mashed sweet potatoes
1,https://www.allrecipes.com/recipe/18057/sweet-...,"1/2 cup butter, melted"
2,https://www.allrecipes.com/recipe/18057/sweet-...,1/3 cup milk
3,https://www.allrecipes.com/recipe/18057/sweet-...,1 cup white sugar
4,https://www.allrecipes.com/recipe/18057/sweet-...,1/2 teaspoon vanilla extract
